In [8]:
# Import libraries
import mysql.connector

In [9]:
def printList(arr):
    for i in arr:
        print(i)

In [10]:
# Establish connection to MySQL and create Database

try:
    db = mysql.connector.connect(host ="localhost",
                                user = "root",
                               password="juancs123")
    c = db.cursor()

    c.execute("CREATE DATABASE seleksi_basdat")

    db.close()
    
except:
    print("Error while connecting to MySQL")


Error while connecting to MySQL


In [11]:
# Use specific database
db = mysql.connector.connect(host="localhost", user="root", password="juancs123", database="seleksi_basdat")

c = db.cursor()
c.execute("ALTER DATABASE seleksi_basdat CHARACTER SET = utf8mb4 COLLATE = utf8mb4_unicode_ci")

In [12]:
# Create tables
product_create = """ CREATE TABLE `seleksi_basdat`.`product` (
    `id` INT NOT NULL AUTO_INCREMENT,
    `listing_id` BIGINT NOT NULL,
    `title` VARCHAR(255) NOT NULL,
    `type` VARCHAR(50) NOT NULL,
    `available` VARCHAR(50),
    `sold` INT,
    `return` VARCHAR(50),
    `seller` VARCHAR(100) NOT NULL,
    `condition` VARCHAR(50) NOT NULL,
    PRIMARY KEY (`id`)
)"""

price_create = """ CREATE TABLE `seleksi_basdat`.`price` (
    `id` INT REFERENCES `product`(`id`),
    `price_original_currency` VARCHAR(10) NOT NULL,
    `price_original_value` FLOAT NOT NULL,
    `price_idr` FLOAT NOT NULL,
    PRIMARY KEY (`id`)
)"""

shipment_create = """ CREATE TABLE `seleksi_basdat`.`shipment` (
    `id` INT REFERENCES `product`(`id`),
    `shipping_availability` BIT NOT NULL,
    `shipping_price_currency` VARCHAR(10),
    `shipping_price_value` FLOAT,
    `shipping_location` VARCHAR(100),
    PRIMARY KEY (`id`)
)"""

c = db.cursor()
c.execute(product_create)
c = db.cursor()
c.execute(price_create)
c = db.cursor()
c.execute(shipment_create)


In [13]:
# import preprocessed data as dataframe
import pandas as pd
import numpy as np
import os

# import the json files
dir = os.getcwd()
product = pd.read_json(dir[:-3]+"data\data_product.json", orient="index")
price = pd.read_json(dir[:-3]+"data\data_price.json", orient="index")
shipment = pd.read_json(dir[:-3]+"data\data_shipment.json", orient="index")


In [14]:
# Insert data to product table
c = db.cursor()

product_insert = """ INSERT INTO `seleksi_basdat`.`product` 
    (`listing_id`, `title`, `type`, `available`, `sold`, `return`, `seller`, `condition`)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""

data = list()

for i in range(len(product)):
    sold_attrib = product.iloc[i,4]
    if ((sold_attrib is not None) and (not pd.isna(sold_attrib))):
        sold_attrib = int(sold_attrib)
    elif (pd.isna(sold_attrib)):
        sold_attrib = None
    tempTuple = (int(product.iloc[i,0]), product.iloc[i,1], product.iloc[i,2], product.iloc[i,3], sold_attrib, product.iloc[i,5], product.iloc[i,6], product.iloc[i,7])
    data.append(tempTuple)


c.executemany(product_insert, data)
db.commit()

# For checking
c = db.cursor()
c.execute(""" SELECT * FROM PRODUCT LIMIT 5""")
printList(c)

(1, 354907151520, 'Microsoft 13" Surface Pro 8 (i5, 8GB RAM, 256GB SSD, Graphite) with Surface Pen', 'Laptops & Netbooks', 'Limited quantity ', 1, '30 days ', 'Always Deals Store', 'New')
(2, 333563280818, 'CHUWI Laptop HeroBook GemiBook CoreBook Pro/Plus/X Intel Core Laptop Windows', 'Laptops & Netbooks', 'Limited quantity ', 32, '30 days ', 'CHUWI Official Store（chuwi2018）', 'New')
(3, 333996205343, 'CHUWI HeroBook Pro 14.1" Windows 10 Laptop Intel N4020 Laptop 8+256GB', 'Laptops & Netbooks', 'Limited quantity ', 14, '30 days ', 'CHUWI Official Store（chuwi2018）', 'New')
(4, 295671743764, 'HP Omen 17.3" Gaming Notebook QHD 165Hz i9-13900HX 32GB RAM 1TB SSD RTX 4090', 'Laptops & Netbooks', 'Limited quantity ', 92, '30 days ', 'Antonline', 'New')
(5, 285185433173, 'Lenovo ThinkPad P1 Gen 5 15.6" 2022 2K 2.3GHz i7-12700H 32GB 1TB SSD - RTX A2000', 'Laptops & Netbooks', 'Limited quantity ', 5, '30 days ', 'ItsWorthMore', 'Excellent - Refurbished')


In [15]:
# Insert data to price table

price_insert = """ INSERT INTO `seleksi_basdat`.`price` 
    (`id`, `price_original_currency`, `price_original_value`, `price_idr`)
    VALUES (%s, %s, %s, %s)
"""

get_product_id = """ SELECT `id` FROM product 
    WHERE product.listing_id = %s """

data = list()

for i in range(len(price)):
    c = db.cursor()
    c.execute(get_product_id, (int(product.iloc[i,0]), ))
    id = c.fetchall()[0][0]

    tempTuple = (id, price.iloc[i,0], price.iloc[i,1], price.iloc[i,2])
    data.append(tempTuple)

c = db.cursor()
c.executemany(price_insert, data)
db.commit()

# For checking
c = db.cursor()
c.execute(""" SELECT * FROM PRICE LIMIT 5""")
printList(c)

(1, 'US $', 669.99, 9999850.0)
(2, 'EUR ', 239.99, 4027000.0)
(3, 'EUR ', 199.99, 3355800.0)
(4, 'US $', 2499.99, 37313300.0)
(5, 'US $', 1399.99, 20895400.0)


In [17]:
# Insert data to shipment table
shipment_insert = """ INSERT INTO `seleksi_basdat`.`shipment` 
    (`id`, `shipping_availability`, `shipping_price_currency`, `shipping_price_value`, `shipping_location`)
    VALUES (%s, %s, %s, %s, %s)
"""

get_product_id = """ SELECT `id` FROM product 
    WHERE product.listing_id = %s """

data = list()

for i in range(len(shipment)):
    c = db.cursor()
    c.execute(get_product_id, (int(product.iloc[i,0]), ))
    id = c.fetchall()[0][0]

    if (shipment.iloc[i,0]):
        availability = 1
    else :
        availability = 0

    ship_price = shipment.iloc[i,2]
    if (pd.isna(ship_price)):
        ship_price = None

    tempTuple = (id, availability, shipment.iloc[i,1], ship_price, shipment.iloc[i,3])
    data.append(tempTuple)


c = db.cursor()
c.executemany(shipment_insert, data)
db.commit()

# For checking
c = db.cursor()
c.execute(""" SELECT * FROM SHIPMENT LIMIT 5""")
printList(c)

(1, 1, 'US $', 54.46, 'Avenel, New Jersey, United States')
(2, 1, None, 0.0, None)
(3, 1, None, 0.0, None)
(4, 0, None, None, 'Atlanta, Georgia, United States')
(5, 1, 'US $', 168.15, 'Sanford, Florida, United States')
